In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import gc
import time
import os

In [3]:
feat_dir = './feat/'
if not os.path.exists(feat_dir):
    os.mkdir(feat_dir)

In [4]:
train_data = pd.read_pickle('./pkl/train_data.pkl')
test_data = pd.read_pickle('./pkl/valid_data.pkl')

data = pd.concat([train_data, test_data]).reset_index(drop=True)

def pidx(p, ps):
    ans = np.nan
    for i, p2 in enumerate(ps):
        if p == p2:
            ans = i
            break
    return ans

from scipy import spatial
def cos_sim(a, b):
    return 1 - spatial.distance.cosine(a, b)

from tqdm import tqdm_notebook

pub_col = ['abstract', 'keywords', 'title', 'venue', 'authors', 'orgs']
for pc in pub_col:
    print(pc)
    print('load data...')
    pub_info = pd.read_pickle('./embed_bert/pub_info_%s.pkl' % pc)
    author_pub_detail = pd.read_pickle('./embed_bert/author_pub_%s.pkl' % pc)
    pub_info.columns = ['paper_id', pc+'_a']
    author_pub_detail.columns = ['author_id', 'paper_ids', pc+'_b']
    data_c = data.merge(pub_info, 'left', 'paper_id').merge(author_pub_detail, 'left', 'author_id')
    print('find idx...')
    data_c['idx'] = data_c.apply(lambda row: pidx(row['paper_id'], row['paper_ids']) if row['label'] == 1 else np.nan, axis=1)
    display(data_c.head(3))
    print('del...')
    for i in tqdm_notebook(range(len(data_c))):
        if pd.isna(data_c.loc[i, 'idx']):
            continue
        v = list(data_c.loc[i, pc+'_b'])
        del v[data_c.loc[i, 'idx'].astype(int)]
        data_c.set_value(i, pc+'_b', v)
    data_c = data_c[[pc+'_a', pc+'_b']]
    display(data_c.head(3))
    print('cal sim...')
    sims = []
    for ca, cb in tqdm_notebook(data_c.values):
        ss = []
        for b in cb:
            cs = cos_sim(ca, b)
            ss.append(cs)
        if len(ss) == 0:
            ss.append(0)
        sims.append(ss)
#     sims_df = pd.DataFrame(sims, columns=[pc+'_sims'])
    sims_df = pd.DataFrame()
    sims_df[pc+'_sims'] = sims
    display(sims_df.head(3))
    print('yi ba suo ha...')
    sims_df[pc+'_sims_'+'min'] = sims_df[pc+'_sims'].apply(np.min)
    sims_df[pc+'_sims_'+'max'] = sims_df[pc+'_sims'].apply(np.max)
    sims_df[pc+'_sims_'+'mean'] = sims_df[pc+'_sims'].apply(np.mean)
    sims_df[pc+'_sims_'+'std'] = sims_df[pc+'_sims'].apply(np.std)
    sims_df[pc+'_sims_'+'mm2'] = (sims_df[pc+'_sims_'+'min'] + sims_df[pc+'_sims_'+'max']) / 2
    display(sims_df.head(3))
    sims_df[[pc+'_sims_'+'min', pc+'_sims_'+'max', pc+'_sims_'+'mean', pc+'_sims_'+'std', pc+'_sims_'+'mm2']].to_pickle('./feat/%s_sims_st_bert.pkl' % (pc))
    del pub_info, author_pub_detail, data_c, sims, sims_df
    gc.collect()

abstract
load data...
find idx...


,author_id,author_name,author_org,label,paper_id,abstract_a,paper_ids,abstract_b,idx
0,8GjtUmBs,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...",[J5KPOi3Y],"[[-0.8647487, 0.53667474, 0.43480504, 0.001089...",NaN
1,EShnTfSe,li_guo,Institute of Pharmacology and Toxicology,1.0,P9a1gcvg,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...",[P9a1gcvg],"[[0.5850733, -0.38639012, 0.59855425, 0.515989...",0.0
2,t1ruuB9N,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...",[T35c9ZAd],"[[1.121394, 0.5555117, 0.9081003, 0.17780448, ...",NaN


del...


,abstract_a,abstract_b
0,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...","[[-0.8647487, 0.53667474, 0.43480504, 0.001089..."
1,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...",[]
2,"[0.5850733, -0.38639012, 0.59855425, 0.5159899...","[[1.121394, 0.5555117, 0.9081003, 0.17780448, ..."


cal sim...


...

,orgs_a,orgs_b
0,"[-0.37271369, -0.5907281, 0.47524145, -0.36693...","[[0.83142114, 0.2809167, 0.21847783, -0.387650..."
1,"[-0.37271369, -0.5907281, 0.47524145, -0.36693...",[]
2,"[-0.37271369, -0.5907281, 0.47524145, -0.36693...","[[0.09807699, -0.5835178, 0.47167218, -0.36723..."


cal sim...


,orgs_sims
0,[0.5488908886909485]
1,[0]
2,[0.5282562375068665]


yi ba suo ha...


,orgs_sims,orgs_sims_min,orgs_sims_max,orgs_sims_mean,orgs_sims_std,orgs_sims_mm2
0,[0.5488908886909485],0.548891,0.548891,0.548891,0.0,0.548891
1,[0],0.000000,0.000000,0.000000,0.0,0.000000
2,[0.5282562375068665],0.528256,0.528256,0.528256,0.0,0.528256
